# Regresión Lineal con Penalización L1 y L2

En este notebook exploraremos las diferencias entre la regresión lineal con penalización L1 (Lasso) y L2 (Ridge).

## Objetivos de aprendizaje:
- Entender cómo L2 (Ridge) maneja variables correlacionadas
- Ver cómo L1 (Lasso) realiza selección de variables
- Comparar los coeficientes resultantes de cada método

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Configuración para reproducibilidad
np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Generación de Datos Sintéticos

Crearemos un dataset con las siguientes características:
- **Variables correlacionadas**: X1, X2, X3 estarán altamente correlacionadas
- **Variables importantes**: X4, X5 tendrán efecto real en Y
- **Variables irrelevantes**: X6, X7, X8, X9, X10 no tendrán efecto en Y

In [ ]:
def generate_illustrative_data(n_samples=60, n_features=50, noise=2.0):
    np.random.seed(42)
    
    # 1. Variables con alta multicolinealidad (X1, X2, X3)
    # Comparten el 99% de su varianza
    shared_signal = np.random.randn(n_samples, 1)
    X_corr = shared_signal + np.random.randn(n_samples, 3) * 0.01
    
    # 2. Variables independientes e importantes
    X_important = np.random.randn(n_samples, 2)
    
    # 3. Variables de ruido puro (muchas más que las importantes)
    X_noise = np.random.randn(n_samples, n_features - 5)
    
    X = np.hstack([X_corr, X_important, X_noise])
    
    # 4. Generar Y (solo las primeras 5 variables tienen impacto)
    # Coeficientes: [10, 10, 10, 5, -5, 0, 0, 0, ...]
    true_beta = np.zeros(n_features)
    true_beta[0:3] = 10.0  # Las correlacionadas
    true_beta[3:5] = [5.0, -5.0] # Las independientes
    
    y = X @ true_beta + np.random.randn(n_samples) * noise
    
    return X, y, true_beta
n_features = 9
X, y, true_coefficients = generate_illustrative_data(n_samples=10, n_features=n_features, noise = 2)

## 2. Exploración de Datos

In [ ]:
X = pd.DataFrame(X, columns=[f"X{i}" for i in  range(n_features)])

In [ ]:
# Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, square=True)
plt.title('Matriz de Correlación de Variables Explicativas', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("Correlaciones entre X1, X2, X3:")
print(correlation_matrix.loc[['X1', 'X2', 'X3'], ['X1', 'X2', 'X3']])

## 3. División y Normalización

Es importante normalizar los datos cuando usamos regularización para que todas las variables estén en la misma escala.

In [ ]:
# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normalización (importante para regularización)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Tamaño del conjunto de entrenamiento: {X_train_scaled.shape}")
print(f"Tamaño del conjunto de prueba: {X_test_scaled.shape}")

## 4. Comparación de Modelos

Entrenaremos tres modelos:
1. **Regresión Lineal Ordinaria** (sin regularización)
2. **Ridge** (penalización L2)
3. **Lasso** (penalización L1)

In [ ]:
# Valores de alpha para experimentar
alpha = 1.0

# Entrenar modelos
lr = LinearRegression()
ridge = Ridge(alpha=alpha)
lasso = Lasso(alpha=alpha)

lr.fit(X_train_scaled, y_train)
ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

# Scores
print("R² Score en conjunto de entrenamiento:")
print(f"  Regresión Lineal: {lr.score(X_train_scaled, y_train):.4f}")
print(f"  Ridge (L2):       {ridge.score(X_train_scaled, y_train):.4f}")
print(f"  Lasso (L1):       {lasso.score(X_train_scaled, y_train):.4f}")

print("\nR² Score en conjunto de prueba:")
print(f"  Regresión Lineal: {lr.score(X_test_scaled, y_test):.4f}")
print(f"  Ridge (L2):       {ridge.score(X_test_scaled, y_test):.4f}")
print(f"  Lasso (L1):       {lasso.score(X_test_scaled, y_test):.4f}")

## 5. Análisis de Coeficientes

Aquí veremos las diferencias clave:
- **L2 (Ridge)** distribuye el peso entre variables correlacionadas
- **L1 (Lasso)** hace selección de variables, llevando algunos coeficientes a exactamente 0

In [ ]:
# Crear DataFrame con los coeficientes
coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Verdadero': list(true_coefficients),
    'Reg. Lineal': lr.coef_,
    'Ridge (L2)': ridge.coef_,
    'Lasso (L1)': lasso.coef_
})

print("Comparación de Coeficientes:")
print(coef_df.to_string(index=False))

# Contar variables con coeficiente cero en Lasso
n_zero_lasso = np.sum(np.abs(lasso.coef_) < 1e-10)
print(f"\nVariables eliminadas por Lasso (coef ≈ 0): {n_zero_lasso}")

## 6. Visualización de Coeficientes

In [ ]:
# Gráfico de barras comparando coeficientes
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = ['Reg. Lineal', 'Ridge (L2)', 'Lasso (L1)']
colors = ['steelblue', 'forestgreen', 'crimson']

for idx, (model, color) in enumerate(zip(models, colors)):
    ax = axes[idx]
    x_pos = np.arange(len(X.columns))
    
    # Coeficientes del modelo
    coefs = coef_df[model].values
    
    # Coeficientes verdaderos como líneas
    true_coefs = coef_df['Verdadero'].values
    
    ax.bar(x_pos, coefs, alpha=0.7, color=color, label='Estimado')
    ax.scatter(x_pos, true_coefs, color='black', s=100, 
               marker='_', linewidths=3, label='Verdadero', zorder=5)
    
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)
    ax.set_xlabel('Variable', fontsize=11)
    ax.set_ylabel('Coeficiente', fontsize=11)
    ax.set_title(f'{model}', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(X.columns, rotation=45)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Observaciones Clave

### Variables Correlacionadas (X1, X2, X3):
- **Regresión Lineal**: Los coeficientes pueden ser inestables debido a la multicolinealidad
- **Ridge (L2)**: Distribuye el peso entre las variables correlacionadas, reduciendo todos los coeficientes pero manteniéndolos no-cero
- **Lasso (L1)**: Tiende a seleccionar una de las variables correlacionadas y eliminar las demás

### Variables Sin Efecto (X6-X10):
- **Regresión Lineal**: Puede asignar coeficientes no-cero por ruido
- **Ridge (L2)**: Reduce los coeficientes pero rara vez los lleva exactamente a cero
- **Lasso (L1)**: Tiende a llevar estos coeficientes exactamente a cero (selección de variables)

### ¿Cuándo usar cada uno?
- **Ridge**: Cuando todas las variables son potencialmente relevantes y hay multicolinealidad
- **Lasso**: Cuando queremos selección de variables automática y un modelo más interpretable

## 8. Experimento: Variando Alpha

Veamos cómo cambian los coeficientes con diferentes valores de alpha (intensidad de la penalización).

In [ ]:
# Rango de alphas
alphas = np.logspace(-3, 2, 50)

ridge_coefs = []
lasso_coefs = []
ridge_mse_train = []
ridge_mse_test = []
lasso_mse_train = []
lasso_mse_test = []

for alpha in alphas:
    ridge_model = Ridge(alpha=alpha)
    lasso_model = Lasso(alpha=alpha, max_iter=10000)
    
    ridge_model.fit(X_train_scaled, y_train)
    lasso_model.fit(X_train_scaled, y_train)
    
    ridge_coefs.append(ridge_model.coef_)
    lasso_coefs.append(lasso_model.coef_)
    
    # Calcular MSE para Ridge
    y_pred_ridge_train = ridge_model.predict(X_train_scaled)
    y_pred_ridge_test = ridge_model.predict(X_test_scaled)
    ridge_mse_train.append(mean_squared_error(y_train, y_pred_ridge_train))
    ridge_mse_test.append(mean_squared_error(y_test, y_pred_ridge_test))
    
    # Calcular MSE para Lasso
    y_pred_lasso_train = lasso_model.predict(X_train_scaled)
    y_pred_lasso_test = lasso_model.predict(X_test_scaled)
    lasso_mse_train.append(mean_squared_error(y_train, y_pred_lasso_train))
    lasso_mse_test.append(mean_squared_error(y_test, y_pred_lasso_test))

ridge_coefs = np.array(ridge_coefs)
lasso_coefs = np.array(lasso_coefs)

# Visualizar paths de coeficientes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Ridge
for i, col in enumerate(X.columns):
    axes[0].plot(alphas, ridge_coefs[:, i], label=col, linewidth=2)
axes[0].set_xscale('log')
axes[0].set_xlabel('Alpha (λ)', fontsize=12)
axes[0].set_ylabel('Coeficiente', fontsize=12)
axes[0].set_title('Ridge (L2): Path de Regularización', fontsize=13, fontweight='bold')
axes[0].axhline(y=0, color='black', linestyle='--', linewidth=0.8)
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(alpha=0.3)

# Lasso
for i, col in enumerate(X.columns):
    axes[1].plot(alphas, lasso_coefs[:, i], label=col, linewidth=2)
axes[1].set_xscale('log')
axes[1].set_xlabel('Alpha (λ)', fontsize=12)
axes[1].set_ylabel('Coeficiente', fontsize=12)
axes[1].set_title('Lasso (L1): Path de Regularización', fontsize=13, fontweight='bold')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=0.8)
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Observa cómo:")
print("- Ridge: Los coeficientes se reducen gradualmente pero nunca llegan exactamente a cero")
print("- Lasso: Los coeficientes llegan a cero en diferentes valores de alpha (selección de variables)")

## 9. Error Cuadrático Medio (MSE) vs Alpha

Analicemos cómo cambia el MSE en los conjuntos de entrenamiento y prueba para diferentes valores de alpha. Esto nos ayudará a identificar:
- **Underfitting**: Cuando alpha es muy grande, ambos errores son altos
- **Overfitting**: Cuando el error de entrenamiento es bajo pero el de prueba es alto
- **Punto óptimo**: Donde el error de prueba es mínimo

In [ ]:
# Visualizar MSE vs Alpha
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Ridge MSE
axes[0].plot(alphas, ridge_mse_train, label='Entrenamiento', linewidth=2.5, 
             color='forestgreen', marker='o', markersize=3)
axes[0].plot(alphas, ridge_mse_test, label='Prueba', linewidth=2.5, 
             color='crimson', marker='s', markersize=3)
axes[0].set_xscale('log')
axes[0].set_xlabel('Alpha (λ)', fontsize=12)
axes[0].set_ylabel('MSE (Error Cuadrático Medio)', fontsize=12)
axes[0].set_title('Ridge (L2): MSE vs Alpha', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Encontrar el alpha óptimo para Ridge
optimal_alpha_ridge = alphas[np.argmin(ridge_mse_test)]
min_mse_ridge = np.min(ridge_mse_test)
axes[0].axvline(x=optimal_alpha_ridge, color='blue', linestyle='--', 
                linewidth=2, alpha=0.7, label=f'Óptimo α={optimal_alpha_ridge:.4f}')
axes[0].legend(fontsize=11)

# Lasso MSE
axes[1].plot(alphas, lasso_mse_train, label='Entrenamiento', linewidth=2.5, 
             color='forestgreen', marker='o', markersize=3)
axes[1].plot(alphas, lasso_mse_test, label='Prueba', linewidth=2.5, 
             color='crimson', marker='s', markersize=3)
axes[1].set_xscale('log')
axes[1].set_xlabel('Alpha (λ)', fontsize=12)
axes[1].set_ylabel('MSE (Error Cuadrático Medio)', fontsize=12)
axes[1].set_title('Lasso (L1): MSE vs Alpha', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

# Encontrar el alpha óptimo para Lasso
optimal_alpha_lasso = alphas[np.argmin(lasso_mse_test)]
min_mse_lasso = np.min(lasso_mse_test)
axes[1].axvline(x=optimal_alpha_lasso, color='blue', linestyle='--', 
                linewidth=2, alpha=0.7, label=f'Óptimo α={optimal_alpha_lasso:.4f}')
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

print("=" * 70)
print("RESULTADOS DE OPTIMIZACIÓN")
print("=" * 70)
print(f"\nRidge (L2):")
print(f"  Alpha óptimo: {optimal_alpha_ridge:.6f}")
print(f"  MSE mínimo (prueba): {min_mse_ridge:.6f}")
print(f"  MSE entrenamiento: {ridge_mse_train[np.argmin(ridge_mse_test)]:.6f}")

print(f"\nLasso (L1):")
print(f"  Alpha óptimo: {optimal_alpha_lasso:.6f}")
print(f"  MSE mínimo (prueba): {min_mse_lasso:.6f}")
print(f"  MSE entrenamiento: {lasso_mse_train[np.argmin(lasso_mse_test)]:.6f}")

print("\n" + "=" * 70)
print("INTERPRETACIÓN")
print("=" * 70)
print("\nObserva que:")
print("- Cuando alpha es muy pequeño → El modelo se ajusta mucho a los datos de entrenamiento")
print("  (MSE de entrenamiento bajo, pero puede haber sobreajuste)")
print("\n- Cuando alpha es muy grande → El modelo está muy regularizado")
print("  (MSE alto en ambos conjuntos, indica subajuste)")
print("\n- El alpha óptimo balancea el sesgo y la varianza del modelo")
print("  (MSE de prueba mínimo)")

## 10. Validación Cruzada: El Método Correcto

El método anterior tiene un problema: **usamos el conjunto de prueba para seleccionar alpha**, lo cual puede introducir sesgo. El método correcto es usar **validación cruzada (cross-validation)** en el conjunto de entrenamiento.

### ¿Por qué validación cruzada?

1. **No contamina el conjunto de prueba**: El conjunto de prueba solo se usa una vez al final para evaluar el modelo final
2. **Más robusto**: Usa múltiples particiones de los datos para estimar el rendimiento
3. **Mejor generalización**: Reduce la varianza en la selección del hiperparámetro

### Métodos disponibles:

- **RidgeCV**: Ridge con validación cruzada integrada
- **LassoCV**: Lasso con validación cruzada integrada

In [ ]:
# Entrenar modelos con validación cruzada
# cv=5 significa 5-fold cross-validation

ridge_cv = RidgeCV(alphas=alphas, cv=5)
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000, random_state=42)

ridge_cv.fit(X_train_scaled, y_train)
lasso_cv.fit(X_train_scaled, y_train)

print("=" * 70)
print("RESULTADOS CON VALIDACIÓN CRUZADA (5-FOLD)")
print("=" * 70)

print(f"\nRidge (L2):")
print(f"  Alpha óptimo (CV): {ridge_cv.alpha_:.6f}")
ridge_pred_test = ridge_cv.predict(X_test_scaled)
ridge_mse_cv = mean_squared_error(y_test, ridge_pred_test)
print(f"  MSE en prueba: {ridge_mse_cv:.6f}")
print(f"  R² en prueba: {ridge_cv.score(X_test_scaled, y_test):.6f}")

print(f"\nLasso (L1):")
print(f"  Alpha óptimo (CV): {lasso_cv.alpha_:.6f}")
lasso_pred_test = lasso_cv.predict(X_test_scaled)
lasso_mse_cv = mean_squared_error(y_test, lasso_pred_test)
print(f"  MSE en prueba: {lasso_mse_cv:.6f}")
print(f"  R² en prueba: {lasso_cv.score(X_test_scaled, y_test):.6f}")

print("\n" + "=" * 70)
print("COMPARACIÓN: Método anterior vs Validación Cruzada")
print("=" * 70)

print(f"\nRidge:")
print(f"  Alpha (método anterior): {optimal_alpha_ridge:.6f}")
print(f"  Alpha (validación cruzada): {ridge_cv.alpha_:.6f}")
print(f"  Diferencia: {abs(optimal_alpha_ridge - ridge_cv.alpha_):.6f}")

print(f"\nLasso:")
print(f"  Alpha (método anterior): {optimal_alpha_lasso:.6f}")
print(f"  Alpha (validación cruzada): {lasso_cv.alpha_:.6f}")
print(f"  Diferencia: {abs(optimal_alpha_lasso - lasso_cv.alpha_):.6f}")

### Comparación de Coeficientes con Validación Cruzada

Veamos cómo cambian los coeficientes al usar el alpha seleccionado por validación cruzada.

In [ ]:
# Crear DataFrame con coeficientes obtenidos por validación cruzada
coef_cv_df = pd.DataFrame({
    'Variable': X.columns,
    'Verdadero': list(true_coefficients),
    'Ridge (CV)': ridge_cv.coef_,
    'Lasso (CV)': lasso_cv.coef_
})

print("Coeficientes con Validación Cruzada:")
print(coef_cv_df.to_string(index=False))

# Contar variables eliminadas por Lasso CV
n_zero_lasso_cv = np.sum(np.abs(lasso_cv.coef_) < 1e-10)
print(f"\nVariables eliminadas por Lasso CV (coef ≈ 0): {n_zero_lasso_cv}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models_cv = ['Ridge (CV)', 'Lasso (CV)']
colors_cv = ['forestgreen', 'crimson']

for idx, (model, color) in enumerate(zip(models_cv, colors_cv)):
    ax = axes[idx]
    x_pos = np.arange(len(X.columns))
    
    coefs = coef_cv_df[model].values
    true_coefs = coef_cv_df['Verdadero'].values
    
    ax.bar(x_pos, coefs, alpha=0.7, color=color, label='Estimado (CV)')
    ax.scatter(x_pos, true_coefs, color='black', s=100, 
               marker='_', linewidths=3, label='Verdadero', zorder=5)
    
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)
    ax.set_xlabel('Variable', fontsize=11)
    ax.set_ylabel('Coeficiente', fontsize=11)
    ax.set_title(f'{model}', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(X.columns, rotation=45)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### Curvas de Validación Cruzada

LassoCV almacena los errores de validación cruzada para cada valor de alpha probado, lo que nos permite visualizar el proceso de selección.

In [ ]:
# LassoCV guarda el MSE promedio de validación cruzada para cada alpha
# mse_path_ tiene forma (n_alphas, n_folds) para cada alpha y fold

# Calcular MSE promedio y desviación estándar a través de los folds
lasso_cv_mean = np.mean(lasso_cv.mse_path_, axis=1)
lasso_cv_std = np.std(lasso_cv.mse_path_, axis=1)

plt.figure(figsize=(12, 6))

# Graficar MSE promedio de CV
plt.plot(lasso_cv.alphas_, lasso_cv_mean, 'b-', linewidth=2.5, label='MSE promedio (5-fold CV)')

# Agregar banda de error (±1 std)
plt.fill_between(lasso_cv.alphas_, 
                 lasso_cv_mean - lasso_cv_std,
                 lasso_cv_mean + lasso_cv_std,
                 alpha=0.2, color='blue', label='±1 desviación estándar')

# Marcar el alpha óptimo
plt.axvline(x=lasso_cv.alpha_, color='red', linestyle='--', 
            linewidth=2, label=f'Alpha óptimo = {lasso_cv.alpha_:.6f}')

plt.xscale('log')
plt.xlabel('Alpha (λ)', fontsize=12)
plt.ylabel('MSE', fontsize=12)
plt.title('Lasso: Curva de Validación Cruzada (5-Fold)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nInterpretación:")
print("-" * 70)
print("- La línea azul muestra el MSE promedio de validación cruzada")
print("- El área sombreada representa la variabilidad entre los folds")
print("- La línea vertical roja indica el alpha que minimiza el MSE de CV")
print("- Este es el método correcto y más robusto para seleccionar alpha")

## Conclusiones

Este ejercicio demuestra las diferencias fundamentales entre L1 y L2, y las mejores prácticas para su aplicación:

### 1. **L2 (Ridge)** es ideal cuando:
- Hay multicolinealidad entre variables
- Todas las variables son potencialmente importantes
- Queremos estabilizar los coeficientes sin eliminar variables
- Ridge reduce gradualmente todos los coeficientes pero rara vez los lleva exactamente a cero

### 2. **L1 (Lasso)** es ideal cuando:
- Queremos selección automática de variables
- Buscamos un modelo interpretable y parsimonioso
- Sospechamos que muchas variables son irrelevantes
- Lasso lleva coeficientes exactamente a cero, eliminando variables del modelo

### 3. **Selección del Hiperparámetro Alpha**:
- **NUNCA usar el conjunto de prueba para seleccionar hiperparámetros** (contamina la evaluación final)
- **Validación cruzada es el método correcto**: usa solo el conjunto de entrenamiento
- Herramientas recomendadas: `RidgeCV` y `LassoCV` de scikit-learn
- Un alpha muy pequeño puede llevar a sobreajuste (overfitting)
- Un alpha muy grande puede llevar a subajuste (underfitting)
- El alpha óptimo balancea sesgo y varianza

### 4. **Flujo de trabajo recomendado**:
1. Dividir datos en entrenamiento y prueba (80/20 o 70/30)
2. Usar validación cruzada en el conjunto de entrenamiento para seleccionar alpha
3. Entrenar el modelo final con el alpha óptimo en todo el conjunto de entrenamiento
4. Evaluar SOLO UNA VEZ en el conjunto de prueba

### 5. **Aplicaciones Prácticas**:
- **Ridge**: Predicción de precios con múltiples características correlacionadas (área, habitaciones, ubicación)
- **Lasso**: Análisis genómico donde solo algunos genes son relevantes entre miles
- **Elastic Net**: Combina L1 y L2 para obtener lo mejor de ambos mundos

### 6. **Recursos adicionales**:
- Documentación de scikit-learn sobre regularización lineal
- "The Elements of Statistical Learning" - Hastie, Tibshirani, Friedman (Capítulo 3)
- Cross-validation en machine learning: [scikit-learn User Guide](https://scikit-learn.org/stable/modules/cross_validation.html)